In [1]:
import cv2
import numpy as np
import math
import pyautogui as p
from skimage.draw import disk
from scipy.ndimage import distance_transform_edt

from skimage.measure import find_contours
from skimage.draw import rectangle
from skimage.color import rgba2rgb,rgb2gray
from skimage.morphology import binary_erosion,binary_opening, binary_dilation, binary_closing,skeletonize, thin
import skimage.io as io
import matplotlib.pyplot as plt
import numpy as np
from skimage.exposure import histogram
from matplotlib.pyplot import bar
from skimage.color import rgb2gray,rgb2hsv

# Convolution:
from scipy.signal import convolve2d
from scipy import fftpack
import math

from skimage.util import random_noise
from skimage.filters import median
from skimage.feature import canny
from skimage.measure import label

In [2]:
def show_images(images,titles=None):
    #This function is used to show image(s) with titles by sending an array of images and an array of associated titles.
    # images[0] will be drawn with the title titles[0] if exists
    # You aren't required to understand this function, use it as-is.
    n_ims = len(images)
    if titles is None: titles = ['(%d)' % i for i in range(1,n_ims + 1)]
    fig = plt.figure()
    n = 1
    for image,title in zip(images,titles):
        a = fig.add_subplot(1,n_ims,n)
        if image.ndim == 2: 
            plt.gray()
        plt.imshow(image)
        a.set_title(title)
        n += 1
    fig.set_size_inches(np.array(fig.get_size_inches()) * n_ims)
    plt.show() 

In [3]:
def countFingers(xa,fingers,xs):
            [x,x1,x2,x3,x4,x5]=xs
            if(xa>x and xa<=x1):
                    fingers[0]=1
            elif(xa>x1 and xa<=x2):
                 fingers[1]=1
            elif(xa>x2 and xa<=x3):
                 fingers[2]=1
            elif(xa>x3 and xa<=x4):
                 fingers[3]=1
            elif(xa>x4 and xa<=x5):
                 fingers[4]=1
            return fingers
def getFingers(img,palm):
    palm=np.where(palm>0.9,1,0)
    psum=np.sum(palm, axis=1)
    y=np.argmax(psum)
    w=np.max(psum)
    i=0
    while(palm[y,i]==0):
        i+=1
    x=i
    f=w/5
    x1=x+f
    x2=x+2*f
    x3=x+3*f
    x4=x+4*f
    x5=x+w
    xs=[x,x1,x2,x3,x4,x5]
    #print(x,w,f)
#img=binary_opening(img)*1
#img=binary_closing(img)*1
    #img=binary_erosion(img)
    #img=binary_erosion(img)
    #img=binary_erosion(img)
    #img=binary_dilation(img)*0.99
    contours=find_contours(img,0.9)
    bounding_boxes=[]
    fingers=[0,0,0,0,0]
    for i in range (0,len(contours)):
        xmin=min(contours[:][i][:,1])
        xmax=max(contours[:][i][:,1])
        ymin=min(contours[:][i][:,0])
        ymax=max(contours[:][i][:,0])
        fw=xmax-xmin
        xa=(xmin+xmax)/2
        aspect=(xmax-xmin)/(ymax-ymin)
        if(aspect<=0.4):
            fingers=countFingers(xa,fingers,xs)
        elif(aspect<=0.7):
            xa2=(xmin+xa)/2
            xa3=(xa+xmax)/2
            fingers=countFingers(xa2,fingers,xs)
            fingers=countFingers(xa3,fingers,xs)
        elif(aspect<=0.9):
            t=(xmax-xa)/3
            xa2=xmin+t
            xa3=xa+2*t
            fingers=countFingers(xa,fingers,xs)
            fingers=countFingers(xa2,fingers,xs)
            fingers=countFingers(xa3,fingers,xs)
        elif(xa>x-f and xa<=x1):
            fingers[0]=1
        elif(aspect<=1.5 and xa>x1 and xa<=x5):
            fingers[4]=1
            fingers[3]=1
            fingers[2]=1
            fingers[1]=1
        bounding_boxes.insert(0,[int(min(contours[:][i][:,1])),int(max(contours[:][i][:,1])),int(min(contours[:][i][:,0])),int(max(contours[:][i][:,0]))])
    img_gray=img
    img_with_boxes=img*0
    print(x,x1,x2,x3,x4,x5)
    for box in bounding_boxes:
        [Xmin, Xmax, Ymin, Ymax] = box
        rr, cc = rectangle(start = (Ymin,Xmin), end = (Ymax,Xmax), shape=img_gray.shape)
        img_with_boxes[rr, cc] = 1 #set color white
    show_images(images=[img_with_boxes],titles=['Image'])
    return fingers

In [4]:
def myApp(img,palm):
    fingers=getFingers(img,palm)
    #print(fingers)
    act=0
    if(np.sum(fingers)==5):
        act=1
    elif((fingers[0] or fingers[1]) and np.sum(fingers)==1):
        act=2
    elif(np.sum(fingers)==0):
        act=3
    elif(fingers[0] and fingers[1] and not(fingers[3]) and not(fingers[4])):
        act=4
    elif((fingers[0] or fingers[1]) and fingers [4] and np.sum(fingers)<=3):
        act=5
    elif((fingers[0] or fingers[1]) and fingers [2]):
        act=6
    else:
        act=0
    print(act)
    return act

In [5]:
 
bgSubThreshold = 50   

# bool, whether the background captured
isBgCaptured = 0   

def removeBG(frame):
    # Computes a foreground mask
    fgmask = bgModel.apply(frame,learningRate=0)
    kernel = np.ones((3, 3), np.uint8)
    fgmask = cv2.erode(fgmask, kernel, iterations=1)
    foreground = cv2.bitwise_and(frame, frame, mask=fgmask)
    return foreground


def calculateFingers(max_contour,blankBG):  # -> finished bool, cnt: finger count
    #  convexity defect
    hull = cv2.convexHull(max_contour, returnPoints=False)
    
    defects = cv2.convexityDefects(max_contour, hull)

    if type(defects) != type(None) and cv2.contourArea(max_contour) < 20000:
        return True , -1

    elif type(defects) != type(None) : 
        cnt = 0    # defects count which its far point angle less than or equal to 90
        for i in range(defects.shape[0]):  # calculate the angle
            s, e, f, d = defects[i][0]   # indices of the points on the contour not the points themselves
            start = tuple(max_contour[s][0])
            end = tuple(max_contour[e][0])
            far = tuple(max_contour[f][0])
            a = math.sqrt((end[0] - start[0]) ** 2 + (end[1] - start[1]) ** 2)
            b = math.sqrt((far[0] - start[0]) ** 2 + (far[1] - start[1]) ** 2)
            c = math.sqrt((end[0] - far[0]) ** 2 + (end[1] - far[1]) ** 2)

            angle = math.acos((b ** 2 + c ** 2 - a ** 2) / (2 * b * c))  # cosine theorem
            if angle <= math.pi / 2 :  # angle less than 90 degree, treat as fingers
                cnt += 1
                cv2.circle(blankBG, far, 8, [255, 0, 0], -1)
        return True, cnt
    return False, 0

whichWay = int(input('which way to use?'))

camera = cv2.VideoCapture(0)
camera.set(10,200)

while camera.isOpened():
    
    ret, frame = camera.read()

    frame = cv2.GaussianBlur(frame, (5, 5), 0)

    # flip the frame horizontally
    if (whichWay == 1):
        frame = cv2.flip(frame, 1)  


    # ROI (Region of interest)
    cv2.rectangle(frame, (320, 0),(640, 384), (255, 0, 0), 2)
    cv2.imshow('original', frame)


    if isBgCaptured == 1: 
        img = removeBG(frame)

        # clip the ROI
        img = img[0:384, 320:640]  
        #cv2.imshow('mask', img)

        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        blur = cv2.GaussianBlur(gray, (35, 35), 0)
        
        # convert the image into binary image
        ret, thresh = cv2.threshold(blur, 70, 255, cv2.THRESH_BINARY)
        cv2.imshow('thresholdImg', thresh)

        # Get the coutours
        thresh1 = thresh.copy()

        # Get distance transformation
        thresh2 = thresh.copy()
        
        if(whichWay == 0):
            contours, hierarchy = cv2.findContours(thresh1, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

            length = len(contours)
            if length > 0:
                max_contour = max(contours, key = lambda x : cv2.contourArea(x))

                # Return 2d array its elements represent the distance bet. each pixel in the FG and the nearest pixel in the BG
                # And i, j represents the pixel location
                distance_transforamtion = distance_transform_edt(thresh2,return_distances = True)    

                # Get the location of the pixel with the maximum distance
                palmi,palmj = np.unravel_index(distance_transforamtion.argmax(),distance_transforamtion.shape)

                # return the distance between the center of the FG and the BG
                radius = distance_transforamtion[palmi, palmj] 

                # Draw disk to get the fingers only from the hand
                radius = radius*1.4
                # rr, cc are arrays of the disk locations (pixels location)
                rr,cc = disk((palmi,palmj), radius,shape=thresh.shape)
                thresh2[rr,cc] = 0 

                cv2.imshow('fingers', thresh2)

                act = myApp(thresh2,thresh)
                if act == 1 :
                    p.press("m")     
                    cv2.putText(frame,"Mute",(50,50), cv2.FONT_HERSHEY_SIMPLEX,2,2)
                elif act == 2:
                    p.press("space")     # start/pause the video
                    cv2.putText(frame,"Play/Pause",(50,50), cv2.FONT_HERSHEY_SIMPLEX,2,2)
                elif act == 3:
                    p.press("up")        # volume up 
                    cv2.putText(frame,"Volume up",(50,50), cv2.FONT_HERSHEY_SIMPLEX,2,2)
                elif act == 4:
                    p.press("down")      # volume down 
                    cv2.putText(frame,"Volume down",(50,50), cv2.FONT_HERSHEY_SIMPLEX,2,2)
                elif act == 5:
                    p.press("right")     # forward the video
                    cv2.putText(frame,"Forward",(50,50), cv2.FONT_HERSHEY_SIMPLEX,2,2)
                elif act == 6:
                    p.press("left")      #backward the video
                    cv2.putText(frame,"Backward",(50,50), cv2.FONT_HERSHEY_SIMPLEX,2,2)

        elif(whichWay == 1):
            contours, hierarchy = cv2.findContours(thresh1, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

            length = len(contours)
            if length > 0:
                
                max_contour = max(contours, key = lambda x : cv2.contourArea(x))
                
                hull = cv2.convexHull(max_contour)
                
                blankBG = np.zeros(img.shape, np.uint8)
                cv2.drawContours(blankBG, [max_contour], 0, (0, 255, 0), 2)
                cv2.drawContours(blankBG, [hull], 0, (0, 0, 255), 3)

                isFinishCal,cnt = calculateFingers(max_contour,blankBG)
                if isFinishCal is True :
                    if cnt == -1 :
                        p.press("m")     
                        cv2.putText(frame,"Mute",(50,50), cv2.FONT_HERSHEY_SIMPLEX,2,2)
                    elif cnt == 0:
                        p.press("space")     # start/pause the video
                        cv2.putText(frame,"Play/Pause",(50,50), cv2.FONT_HERSHEY_SIMPLEX,2,2)
                    elif cnt == 1:
                        p.press("up")        # volume up 
                        cv2.putText(frame,"Volume up",(50,50), cv2.FONT_HERSHEY_SIMPLEX,2,2)
                    elif cnt == 2:
                        p.press("down")      # volume down 
                        cv2.putText(frame,"Volume down",(50,50), cv2.FONT_HERSHEY_SIMPLEX,2,2)
                    elif cnt == 3:
                        p.press("right")     # forward the video
                        cv2.putText(frame,"Forward",(50,50), cv2.FONT_HERSHEY_SIMPLEX,2,2)
                    elif cnt == 4:
                        p.press("left")      #backward the video
                        cv2.putText(frame,"Backward",(50,50), cv2.FONT_HERSHEY_SIMPLEX,2,2)
                cv2.imshow('output', blankBG)
        cv2.imshow('gesture', frame)

    
    # wait for keyboard pressed key 
    k = cv2.waitKey(10)
    if k == 27:  # press ESC to exit
        camera.release()
        cv2.destroyAllWindows()
        break
    elif k == ord('b'):  # press 'b' to capture the background
        bgModel = cv2.createBackgroundSubtractorMOG2(0, bgSubThreshold)
        isBgCaptured = 1
